In [16]:
data = [i.strip('\n').split('\t') for i in open('/Users/Diego/Documents/School/ISTA539/final-project-dalcantara7/openclass_prize.train')]
print(data[0])
print(" ")
print(data[1])

['What is the Turing Test?', 'The original Turing test involved three entities communicating with each other through computer terminals in separate rooms - a computer program trying to pass itself off as a human, a human trying to prove that she or he is human, and a human judge deciding which of the competitors is human and which a machine. This is different from the popular idea that it\'s about a human judging whether one subject is human or machine. The reason it\'s different is that Turing\'s original proposal was not intended as a practical test, but rather as a thought experiment making a philosophical argument - that it is impossible to determine whether something is "intelligent" better than by judging its linguistic interactions, and that therefore a machine that sounds intelligent has to be considered truly intelligent. And by "intelligent," Turing meant, like a human, with consciousness.', 'Which of the following is NOT an objection to the Turing Test mentioned in the readi

In [18]:
X = []
Y = []
for i in range(len(data)):
    if i % 2 == 0:
        X.append(' '.join(data[i]))
    else:
        Y.append(data[i])

print(X[0:3])
print(" ")
print(Y[0:5])

['What is the Turing Test? The original Turing test involved three entities communicating with each other through computer terminals in separate rooms - a computer program trying to pass itself off as a human, a human trying to prove that she or he is human, and a human judge deciding which of the competitors is human and which a machine. This is different from the popular idea that it\'s about a human judging whether one subject is human or machine. The reason it\'s different is that Turing\'s original proposal was not intended as a practical test, but rather as a thought experiment making a philosophical argument - that it is impossible to determine whether something is "intelligent" better than by judging its linguistic interactions, and that therefore a machine that sounds intelligent has to be considered truly intelligent. And by "intelligent," Turing meant, like a human, with consciousness. Which of the following is NOT an objection to the Turing Test mentioned in the reading? In

In [29]:
from nltk.stem.snowball import SnowballStemmer
import re

def clean_data(dataset):
    clean_data = []
    sno = SnowballStemmer('english')

    for entry in dataset:
        lowered = entry.lower()
        sans_special_chars = re.sub(r'\W',' ', lowered)
        sans_extra_spaces = re.sub(r'\s+',' ', sans_special_chars)
        sans_underscore = sans_extra_spaces.replace('_', ' ')
        split = sans_underscore.split()
        stemmed = ' '.join([sno.stem(word) for word in split])
        clean_data.append(stemmed)

    return clean_data

In [30]:
clean_X = clean_data(X)
print(clean_X[0:3])

['what is the ture test the origin ture test involv three entiti communic with each other through comput termin in separ room a comput program tri to pass itself off as a human a human tri to prove that she or he is human and a human judg decid which of the competitor is human and which a machin this is differ from the popular idea that it s about a human judg whether one subject is human or machin the reason it s differ is that ture s origin propos was not intend as a practic test but rather as a thought experi make a philosoph argument that it is imposs to determin whether someth is intellig better than by judg it linguist interact and that therefor a machin that sound intellig has to be consid truli intellig and by intellig ture meant like a human with conscious which of the follow is not an object to the ture test mention in the read intellig is not the same thing as conscious mani human be might not pass the ture test languag is not necessarili the onli way to demonstr high intell

In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
print("Length of X: ", len(X))
print("Length of Y: ", len(Y))
print("Length of X_train: ", len(X_train))
print("Length of Y_train: ", len(Y_train))

Length of X:  163
Length of Y:  163
Length of X_train:  130
Length of Y_train:  130


In [34]:
from keras.preprocessing.text import Tokenizer
import keras.preprocessing.sequence as kps
NUM_WORDS = 120000

tokenizer = Tokenizer(num_words=NUM_WORDS)
tokenizer.fit_on_texts(X_train)

train_indices = tokenizer.texts_to_sequences(X_train)
word_index = tokenizer.word_index
train_indices = kps.pad_sequences(train_indices, padding='post')

test_indices = tokenizer.texts_to_sequences(X_test)
test_indices = kps.pad_sequences(test_indices, padding='post', maxlen=len(train_indices[0]))

print("Train Data: ", train_indices[0:3])
print("Test Data: ", test_indices[0:3])

Train Data:  [[  51   12   52 ...    0    0    0]
 [1102 2689   29 ...    0    0    0]
 [ 384    1  301 ...    0    0    0]]
Test Data:  [[  51    7    4 ...    0    0    0]
 [  51    7    4 ...    0    0    0]
 [  51    7 1128 ...    0    0    0]]


In [ ]:
from keras import models, layers
EMBEDDING_DIM = 300
embeddings_index = {}

f = open("graduate-project-dalcantara7/glove.twitter.27B/glove.twitter.27B.200d.txt")
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

embedding_layer = layers.Embedding(len(word_index) + 1,
                        EMBEDDING_DIM,
                        weights=[embedding_matrix],
                        input_length=len(train_indices[0]),
                        trainable=True,
                        mask_zero=True)

In [37]:
flat_Y = [item for sublist in Y for item in sublist]
print(len(flat_Y))

716
